<div class="usecase-title">Urban Heat Island Effect Reduction</div>

<div class="usecase-authors"><b>Authored by: </b> Amy Tran & Siyu Ai</div>

<div class="usecase-duration"><b>Duration:</b> 60 mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>Intermediate</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>Python</div>
</div>

<div class="usecase-section-header">Scenario</div>

As a city planner, I would like to understand the effect of urban heat island arising from business activities and energy consumption by looking into the distribution of trees vs projected energy consumption across the City of Melbourne.

As a city planner, I would like to have a system that could help me identify areas of highest greening priority across the City of Melbourne .

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- Load relevant datasets required for analysis
- Map and calculate the number of trees and total projected energy consumption for each laneways area and its surrounding
- Understand how greening priority scores are computed
- Visualise greening priority scores of laneways area on a heatmap

<div class="usecase-section-header">Python Libraries</div>

In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import plotly.express as px
import geopy.distance

#pip install plotly==5.8.0
#pip install geopy

<div class="usecase-section-header">Data Extraction</div>

In [6]:
def get_data(base, url, size = 0):
    target_filters = f'records?limit={10}&offset={size}&timezone=UTC'
    target_url = f'{base}{url}/{target_filters}'
    result = session.get(target_url+f'&apikey={API_KEY}')
    status_code = result.status_code
    if status_code == 200:
        result_json = result.json()
        max_results = result_json['total_count']
        links = result_json['links']
        records = result_json['records']
        records_df = pd.json_normalize(records)
    
        #Update column labels
        records_df.drop(columns=['links'],inplace=True)
        column_names = records_df.columns.values.tolist()

        #Replace geolocation.lat & geolocation.lon
        column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='geolocation' else a for a in column_names]
        column_names = [i.split('.')[-1] for i in column_names]
        records_df.columns = column_names
    
        next_url = None
             
        #Obtain next url
        if records_df.shape[0] != max_results:
            for l in links:
                if l['rel'] == 'next':
                    next_url = l['href']
    
        return[records_df, next_url, column_names, max_results, status_code]
    else: return[None, None, None, None, status_code]

In [11]:
#Extract laneways-with-greening-potential data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'laneways-with-greening-potential'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
laneways = data.copy()
#Rename Longitude and Latitude columns
laneways = laneways.rename(columns = {'geo_point_2d.lon' : 'lon', 'geo_point_2d.lat' : 'lat'})
print('Completed:', laneways.shape[0], 'rows,', laneways.shape[1], 'columns extracted' )

Completed: 246 rows, 26 columns extracted


In [12]:
#Extract urban forest data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'trees-with-species-and-dimensions-urban-forest'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
uf = data.copy()
#Rename Longitude and Latitude columns
uf = uf.rename(columns = {'coordinatelocation.lon' : 'lon', 'coordinatelocation.lat' : 'lat'})
print('Completed:', uf.shape[0], 'rows,', uf.shape[1], 'columns extracted' )

Completed: 76928 rows, 22 columns extracted


In [40]:
#Extract energy consumption projection data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'block-level-energy-consumption-modelled-on-building-attributes-2026-projection-b'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
energy = data.copy()
#Rename Longitude and Latitude columns
energy = energy.rename(columns = {'geo_point_2d.lon' : 'lon', 'geo_point_2d.lat' : 'lat'})
print('Completed:', energy.shape[0], 'rows,', energy.shape[1], 'columns extracted' )

Completed: 628 rows, 9 columns extracted


<div class="usecase-section-header">Calculate number of trees vs projected energy consumption</div>

In [15]:
#Create function to calculate distance between 2 coordinates
import geopy.distance
def distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    km = geopy.distance.geodesic(coords_1, coords_2).km
    return (km)

In [16]:
#Count number of trees in each greening location and its surrounding within the radius set
#Set radius
radius = 0.1 #KM
location_count = len(laneways)

#Filter Urban Forest dataset to include only trees within the bounding box
#Create bounding box
#Find Min and Max latitude and longitude of greening location
max_lon = laneways['lon'].max()
max_lat = laneways['lat'].max()
min_lon = laneways['lon'].min()
min_lat = laneways['lat'].min()

#Find coordinate of bounding box 
b = [45, 135, 225, 315]
d = radius

#North East (NE) coordinate of bounding box
origin = geopy.Point(max_lat, max_lon)
destination = geopy.distance.geodesic(kilometers=d).destination(origin, b[0])
NE_lat, NE_lon = destination.latitude, destination.longitude

#South East (SE) coordinate of bounding box
origin = geopy.Point(min_lat, max_lon)
destination = geopy.distance.geodesic(kilometers=d).destination(origin, b[1])
SE_lat, SE_lon = destination.latitude, destination.longitude

#South West (SW) coordinate of bounding box
origin = geopy.Point(min_lat, min_lon)
destination = geopy.distance.geodesic(kilometers=d).destination(origin, b[2])
SW_lat, SW_lon = destination.latitude, destination.longitude

#North West (NW) coordinate of bounding box
origin = geopy.Point(max_lat, min_lon)
destination = geopy.distance.geodesic(kilometers=d).destination(origin, b[3])
NW_lat, NW_lon = destination.latitude, destination.longitude

#Latitude boundaries
max_lat_bound = max(NW_lat, NE_lat)
min_lat_bound = min(SW_lat, SE_lat)
#Longitude boundaries
max_lon_bound = max(NE_lon, SE_lon)
min_lon_bound = min(NW_lon, SW_lon)

#Filter Urban Forest dataset to include only trees within the bounding box
uf_df = uf.loc[(uf['lat'].between(min_lat_bound, max_lat_bound)) & (uf['lon'].between(min_lon_bound, max_lon_bound))]
uf_df = uf_df.reset_index()

#Filter Energy Consumption Projection dataset to include only sites within the bounding box
energy_df = energy.loc[(energy['lat'].between(min_lat_bound, max_lat_bound)) & (energy['lon'].between(min_lon_bound, max_lon_bound))]
energy_df = energy_df.reset_index()

#Count number of trees and sum up projected energy consumption in each greening location and its surrounding within the radius set
for i in range(location_count):
#for i in range(2):
    #Longitude and Latitude of greening area
    green_lon = laneways.loc[i, 'lon']
    green_lat = laneways.loc[i, 'lat']
    #Count number of trees within set radius
    tree_count = 0
    for x in range(len(uf_df)):
        tree_lon = uf_df.loc[x, 'lon']
        tree_lat = uf_df.loc[x, 'lat']
        km = distance(tree_lat, tree_lon, green_lat , green_lon)
        if abs(km) < radius:
            tree_count = tree_count 
            tree_count = tree_count + 1
    laneways.loc[i, 'tree_count'] = tree_count
    #Total projected energy consumption within set radius
    total_energy = 0
    for k in range(len(energy_df)):
        site_lon = energy_df.loc[k, 'lon']
        site_lat = energy_df.loc[k, 'lat']
        site_total = energy_df.loc[k, 'total']
        site_km = distance(site_lat, site_lon, green_lat , green_lon)
        if abs(site_km) < radius:
            total_energy = total_energy
            total_energy = total_energy + site_total
    laneways.loc[i, 'total_energy'] = total_energy
    print('Location:', i + 1, 'out of', location_count, ', Latitude:', green_lat, ', Longitude:', green_lon, ', Number of trees:', tree_count, ', Total energy consumption projected:', total_energy)

print('Completed')

Location: 1 out of 246 , Latitude: -37.81980050254932 , Longitude: 144.9623447558369 , Number of trees: 29 , Total energy consumption projected: 0.0
Location: 2 out of 246 , Latitude: -37.81097635318543 , Longitude: 144.97170834375964 , Number of trees: 41 , Total energy consumption projected: 39573.5186276
Location: 3 out of 246 , Latitude: -37.81111151578862 , Longitude: 144.97244046374837 , Number of trees: 35 , Total energy consumption projected: 16389.2099678
Location: 4 out of 246 , Latitude: -37.81161232134716 , Longitude: 144.97077449834737 , Number of trees: 54 , Total energy consumption projected: 16389.2099678
Location: 5 out of 246 , Latitude: -37.8124734926234 , Longitude: 144.9713761243581 , Number of trees: 86 , Total energy consumption projected: 21433.0537357
Location: 6 out of 246 , Latitude: -37.81304184241003 , Longitude: 144.97240833983707 , Number of trees: 69 , Total energy consumption projected: 46318.9282513
Location: 7 out of 246 , Latitude: -37.81410455066066

In [26]:
# Visualisation 1: Heatmap to show projected energy consumption with number of trees 
fig = px.density_mapbox(laneways, lat = 'lat', lon = 'lon', z = 'total_energy',mapbox_style="stamen-terrain", radius= 40, opacity = 0.5, 
        title = 'Heatmap showing projected energy consumption with number of trees (on hover) across City of Melbourne', 
        width = 1000, height = 800, zoom = 14, hover_data= ['total_energy', 'tree_count'],
        color_continuous_scale= [
                [0.0, "green"],
                [0.5, "green"],
                [0.51, "yellow"],
                [0.71, "yellow"],
                [0.72, "red"],
                [1, "red"]])
fig

<div class="usecase-section-header">Greening Priority Score</div>

In [33]:
# Divide total_energy by trees_within_radius to get energy per tree
laneways['energy_per_tree'] = laneways['total_energy'] / laneways['tree_count']

In [34]:
#Convert potential rank into actual number
laneways['farm_rank'] = laneways['farm_rank'].str.lower()
laneways['farm_rank'] = laneways['farm_rank'].replace({'lowest potential': 0, 
                                                       'some potential': 1, 
                                                       'good potential': 2, 
                                                       'highest potential': 3})

laneways['vert_rank'] = laneways['vert_rank'].str.lower()
laneways['vert_rank'] = laneways['vert_rank'].replace({'lowest potential': 0, 
                                                       'some potential': 1, 
                                                       'good potential': 2, 
                                                       'highest potential': 3})


laneways['fores_rank'] = laneways['fores_rank'].str.lower()
laneways['fores_rank'] = laneways['fores_rank'].replace({'Lowest potential': 0, 
                                                         'some potential': 1, 
                                                         'good potential': 2, 
                                                         'highest potential': 3})

laneways['park_rank'] = laneways['park_rank'].str.lower()
laneways['park_rank'] = laneways['park_rank'].replace({'lowest potential': 0, 
                                                       'some potential': 1, 
                                                       'good potential': 2, 
                                                       'highest potential': 3})

laneways['total_rank'] = laneways[['farm_rank', 'vert_rank', 'fores_rank', 'park_rank']].sum(axis=1)

In [37]:
#Compute greening priority and put on 0-100 scale
laneways['greening_priority'] = laneways['energy_per_tree']*laneways['total_rank']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
laneways['greening_priority'] = scaler.fit_transform(laneways['greening_priority'].values.reshape(-1, 1))
laneways['greening_priority'] = round(laneways['greening_priority'] * 100,1)

In [38]:
#Visualisation 2: Heatmap to show laneways with greening priotity
fig = px.density_mapbox(laneways, lat = 'lat', lon = 'lon', z = 'greening_priority',mapbox_style="stamen-terrain", radius= 40, opacity = 0.5, 
        title = 'Heatmap showing laneways with greening priotity across City of Melbourne', 
        width = 1000, height = 800, zoom = 14, hover_data= ['total_energy', 'tree_count'], hover_name = 'greening_priority',
        color_continuous_scale= [
                [0.0, "lightgreen"],
                [0.2, "lightgreen"],
                [0.21, "green"],
                [0.50, "green"],
                [0.51, "darkgreen"],
                [1, "darkgreen"]])
fig

<div class="usecase-section-header">References</div>

City of Melbourne Open Data, 'Laneways with greening potential', City of Melbourne, date retrieved 17 May 2023, https://data.melbourne.vic.gov.au/explore/dataset/laneways-with-greening-potential/

City of Melbourne Open Data, 'Trees, with species and dimensions (Urban Forest)', City of Melbourne, date retrieved 17 May 2023, https://melbournetestbed.opendatasoft.com/explore/dataset/trees-with-species-and-dimensions-urban-forest/

City of Melbourne Open Data, 'Block level energy consumption (modelled on building attributes) - 2026 projection - business-as-usual scenario', City of Melbourne, date retrieved 17 May 2023, 
https://data.melbourne.vic.gov.au/explore/dataset/block-level-energy-consumption-modelled-on-building-attributes-2026-projection-b/information/
